In [4]:

import collections as col
import xmlrpc.client as client
import csv as csv
import json as js

DEEPBLUEURL = 'http://deepblue.mpi-inf.mpg.de/xmlrpc'
USERKEY = 'secret_key'

uk = USERKEY

mdfile = '/home/pebert/work/code/mpggit/creepiest/datasrc/encode/20160211_ENCODE_metadata.tsv'
datasets_file = '/home/pebert/work/code/mpggit/creepiest/datasrc/encode/20160223_ENCODE_sample_datasets.json'

proxy = client.ServerProxy(DEEPBLUEURL, encoding="utf-8", allow_none=True)
print(proxy.echo(uk))


sample_col = col.defaultdict(set)
with open(mdfile, 'r') as infile:
    rows = csv.DictReader(infile, delimiter='\t')
    for r in rows:
        if not r['Biosample treatments'].strip():
            s, r = proxy.search(r['File accession'], 'experiments', uk)
            assert s == 'okay', 'Error: {}'.format(r)
            try:
                expid = r[0][0]
            except IndexError:  # empty result
                continue
            s, r = proxy.info(expid, uk)
            assert s == 'okay', 'Error: {}'.format(r)
            smpid = r[0]['sample_id']
            asmbl = r[0]['genome']
            s, r = proxy.list_experiments(asmbl, '', '', '', smpid, '', 'ENCODE', uk)
            assert s == 'okay', 'Error: {}'.format(r)
            for expinfo in r:
                sample_col[smpid].add(tuple(expinfo))

# serializing the defaultdict of sets of tuples results in TypeError because
# JSON has no concept of tuples (just lists)

convdict = dict()
for k, val in sample_col.items():
    convdict[k] = sorted([list(v) for v in val])
                
with open(datasets_file, 'w') as outfile:
    js.dump(convdict, outfile, sort_keys=True, indent=1)


['okay', 'DeepBlue (1.6.7) says hi to Peter Ebert']
